<a href="https://colab.research.google.com/github/cnovak232/DL_Speech_Enhancement/blob/main/DL_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load Data by cloning the repo - easiest way to access shared data

In [ ]:
!git clone https://github.com/cnovak232/DL_Speech_Enhancement.git


In [ ]:
!pip install PyWavelets

Define some helper function for plotting and playing audio

In [ ]:
import torch
import torchaudio as ta
import librosa as lib
from IPython.display import Audio, display
import matplotlib
import matplotlib.pyplot as plt
import pywt

# helper functions for audio and what not
# mostly taken for torchaudio tutorials 

def play_audio(waveform, sample_rate):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    if num_channels == 1:
        display(Audio(waveform[0], rate=sample_rate))
    elif num_channels == 2:
        display(Audio((waveform[0], waveform[1]), rate=sample_rate))
    else:
        raise ValueError("Waveform with more than 2 channels are not supported.")

def plot_waveform(waveform, sample_rate, title="Waveform", xlim=None, ylim=None):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sample_rate

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].plot(time_axis, waveform[c], linewidth=1)
        axes[c].grid(True)
    if num_channels > 1:
        axes[c].set_ylabel(f'Channel {c+1}')
    if xlim:
        axes[c].set_xlim(xlim)
    if ylim:
        axes[c].set_ylim(ylim)

    figure.suptitle(title)
    plt.show(block=False)

def get_spectrogram(
    waveform = None,
    n_fft = 512,
    win_len = None,
    hop_len = None,
    power = 1.0 ):
    spectrogram = ta.transforms.Spectrogram(
      n_fft=n_fft,
      win_length=win_len,
      hop_length=hop_len,
      center=True,
      pad_mode="reflect",
      power=power )
    
    return spectrogram(waveform)

def plot_spectrogram(spec, type = "amplitude", title=None, ylabel='freq_bin', aspect='auto', xmax=None):
    fig, axs = plt.subplots(1, 1)
    axs.set_title(title or 'Spectrogram (db)')
    axs.set_ylabel(ylabel)
    axs.set_xlabel('frame')
    toDb = ta.transforms.AmplitudeToDB(type)
    im = axs.imshow(toDb(spec), origin='lower', aspect=aspect)
    if xmax:
        axs.set_xlim((0, xmax))
    fig.colorbar(im, ax=axs)
    plt.show(block=False)


class DWT(object):
    def __init__(self, name, level):
        self.name = name
        self.level = level

    def __call__(self, x):
        coeffs = pywt.wavedec(x, self.name, level=self.level,mode='per')
        xdwt = torch.empty(0)
        for band in coeffs:
            torch.from_numpy(band)
            xdwt = torch.cat( (xdwt, torch.from_numpy(band)), dim=1)
        return xdwt

class IDWT(object):
    def __init__(self, name, level):
        self.name = name
        self.level = level

    def __call__(self, xdwt):
        length = xdwt.shape[1]
        coeffs = []
        for idx in range(self.level):
            hlen = length // 2
            coeffs.append( xdwt[:,hlen:length].numpy() )
            length = hlen

        coeffs.append( xdwt[:,:length].numpy() )
        coeffs.reverse()

        xr = pywt.waverec(coeffs, self.name, mode='per')
        return torch.from_numpy(xr) 

def norm_spec( spec ):
    return spec, 1
    #normed = spec / spec.max()
    #return normed, spec.max()


Define a Custom Dataset class for the Data and read it in

In [ ]:
# read dataset in and downsample / transform / pad if needed
from torch.utils.data import Dataset
import torch
import os

class VoiceBankDemand(Dataset):
    def __init__(self, clean_dir, noisy_dir, list_dir, 
                 data = "train", len_samples = None, downsample = None, 
                 transform = None ):
        self.clean_dir = clean_dir
        self.noisy_dir = noisy_dir
        self.list_dir = list_dir
        self.num_samples = len_samples
        self.downsample = downsample
        self.transform = transform
        self.data = data
    
    def __len__(self):
        return len(self.list_dir)

    def __getitem__( self, idx ):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        clean_name = os.path.join( self.clean_dir, self.list_dir[idx] )
        noisy_name = os.path.join( self.noisy_dir, self.list_dir[idx] )
        clean_audio, fs = ta.load(clean_name)
        noisy_audio, fs= ta.load(noisy_name)

        if self.downsample:
            downsampler = ta.transforms.Resample(fs,self.downsample)
            clean_audio = downsampler( clean_audio )
            noisy_audio = downsampler( noisy_audio )

        orig_len = clean_audio.shape[1]

        if self.num_samples:
            # trim or pad audio to fixed length 
            if clean_audio.shape[1] > num_samples:
                clean_audio = clean_audio[:,:num_samples]
                noisy_audio = noisy_audio[:,:num_samples]
            elif clean_audio.shape[1] < num_samples:
                pad_len = int( num_samples - clean_audio.shape[1] )
                pad = torch.zeros(1,pad_len)
                clean_audio = torch.cat((clean_audio,pad), dim=1)
                noisy_audio = torch.cat((noisy_audio,pad),dim=1)
        
        if self.data == "test":
            if self.transform:
                noisy_audio = self.transform( noisy_audio )
                clean_audio = self.transform( clean_audio )
                #clean_mag,_ = torch.abs(clean_trnsfrm) 
                #clean_audio = (clean_audio, clean_mag)
               # noisy_mag, norm_val = torch.abs(noisy_trnsfrm)
                #noisy_phase = torch.angle(noisy_trnsfrm)
                #noisy_audio = (noisy_mag, noisy_phase, noisy_audio, norm_val)
        else:
            if self.transform:
                clean_audio  = self.transform( clean_audio )
                noisy_audio  = self.transform( noisy_audio ) 

        sample = (clean_audio, noisy_audio, orig_len)

        return sample

train_clean_path = './DL_Speech_Enhancement/clean_trainset_28spk_wav'
train_noisy_path = './DL_Speech_Enhancement/noisy_trainset_28spk_wav'
test_clean_path  = './DL_Speech_Enhancement/clean_testset_wav'
test_noisy_path  = './DL_Speech_Enhancement/noisy_testset_wav'

list_dir_train = os.listdir(train_clean_path)
list_dir_train.sort()
list_dir_test = os.listdir(test_clean_path)
list_dir_test.sort()

target_fs = 8000 # downsample to 16 KHz
spectrogram = ta.transforms.Spectrogram(
    n_fft=320, # size for the CRNN
    power=1.0,
    normalized = False )

WAVE_UNET_LEN = 16384 # length the WaveUNET processing
num_samples = WAVE_UNET_LEN * 3


wavelet = DWT( name = 'db8', level = 5 )


# return turns a tuple of ( clean_data,noisy_data,original_length)
# data type depends on if transform was specified
train_set = VoiceBankDemand( clean_dir = train_clean_path,
                             noisy_dir = train_noisy_path,
                             list_dir = list_dir_train,
                             len_samples = None, # clip or pad samples to be 5s
                             downsample = target_fs, # downsample to 16Khz
                             transform = spectrogram )

# returns tuples within tuples: clean_data, noisy_data, original length
# If using a transform (like spectrogram) the subtuples will be:
# clean_data: (clean_waveform, clean_spec)
# noisy_data: ( noisy_spec, noisy_phase, noisy_waveform, normalization_val )
test_set = VoiceBankDemand( clean_dir = test_clean_path,
                            noisy_dir = test_noisy_path,
                            list_dir = list_dir_test,
                            data = "test",
                            len_samples = None,
                            downsample = target_fs,
                            transform = spectrogram )

#############################################################
# for viewing samples and such
clean, noisey, orgin_len = train_set[1]

#transform = DWT( name = 'db4', level = 3 )
#inv_transform = IDWT( name = 'db4', level = 3)
#clean_dwt = transform(clean)


#cleanr = inv_transform(clean_dwt)

#plot_waveform(clean_dwt,target_fs)
#plot_waveform(cleanr,target_fs)

clean_test, noisy_test, orig_len = test_set[0]

#noisy_mag, noisy_phase, noisy_audio, norm_val = noisy_test

plot_spectrogram(clean.squeeze())
plot_spectrogram(noisey.squeeze())

#play_audio(noisy_audio[:,:orig_len],target_fs)

In [ ]:
# DOESN"T WORK WELL, DO NOT USE
import torch
import torch.nn as nn

# U-Net Style Autoencoder from to start from
# https://medium.com/@sriskandaryan/autoencoders-demystified-audio-signal-denoising-32a491ab023a

class UNet(nn.Module):
    def __init__(self, chnls_in=1, chnls_out=1):
        super(UNet, self).__init__()
        self.down_conv_layer_1 = DownConvBlock(chnls_in, 64, norm=False)
        self.down_conv_layer_2 = DownConvBlock(64, 128)
        self.down_conv_layer_3 = DownConvBlock(128, 256)
        self.down_conv_layer_4 = DownConvBlock(256, 256, dropout=0.5)
        self.down_conv_layer_5 = DownConvBlock(256, 256, dropout=0.5)
        self.down_conv_layer_6 = DownConvBlock(256, 256, dropout=0.5)

        self.up_conv_layer_1 = UpConvBlock(256, 256, kernel_size=(2,3), stride=2, padding=0, dropout=0.5)# 256+256 6 5 kernel_size=(2, 3), stride=2, padding=0
        self.up_conv_layer_2 = UpConvBlock(512, 256, kernel_size=(2,3), stride=2, padding=0, dropout=0.5) # 256+256 1 4
        self.up_conv_layer_3 = UpConvBlock(512, 256, kernel_size=(2,3), stride=2, padding=0, output_padding = 2, dropout=0.5) # 2 3
        self.up_conv_layer_4 = UpConvBlock(512, 128, dropout=0.5) # 3 2
        self.up_conv_layer_5 = UpConvBlock(256, 64) # 4 1
        self.up_conv_layer_6 = UpConvBlock(512, 128)
        self.up_conv_layer_7 = UpConvBlock(256, 64)
        self.upsample_layer = nn.Upsample(scale_factor=2)
        self.zero_pad = nn.ZeroPad2d((1, 0, 1, 0))
        self.conv_layer_1 = nn.Conv2d(128, chnls_out, 4, padding='same')
        self.activation = nn.Tanh()
    
    def forward(self, x):
        enc1 = self.down_conv_layer_1(x)
        enc2 = self.down_conv_layer_2(enc1) 
        enc3 = self.down_conv_layer_3(enc2)
        enc4 = self.down_conv_layer_4(enc3)
        enc5 = self.down_conv_layer_5(enc4)
        enc6 = self.down_conv_layer_6(enc5)
 
        dec1 = self.up_conv_layer_1(enc6, enc5)
        dec2 = self.up_conv_layer_2(dec1, enc4)
        dec3 = self.up_conv_layer_3(dec2, enc3)
        dec4 = self.up_conv_layer_4(dec3, enc2)
        dec5 = self.up_conv_layer_5(dec4, enc1)

        final = self.upsample_layer(dec5)
        final = self.zero_pad(final)
        final = self.conv_layer_1(final)
        return final

class UpConvBlock(nn.Module):
    def __init__(self, ip_sz, op_sz, kernel_size=4, stride= 2, padding=0 , output_padding = 0, dropout=0.0):
        super(UpConvBlock, self).__init__()
        self.layers = nn.ModuleList([
            nn.ConvTranspose2d(ip_sz, op_sz, kernel_size=kernel_size, stride=stride, padding=padding, output_padding = output_padding),
            nn.InstanceNorm2d(op_sz),
            nn.ReLU(),
        ])
        if dropout:
            self.layers += [nn.Dropout(dropout)]
    def forward(self, x, enc_ip):
        x = nn.Sequential(*(self.layers))(x)
        op = torch.cat((x, enc_ip), 1)
        return op


class DownConvBlock(nn.Module):
    def __init__(self, ip_sz, op_sz, kernel_size=4, norm=True, dropout=0.0):
        super(DownConvBlock, self).__init__()
        self.layers = nn.ModuleList([nn.Conv2d(ip_sz, op_sz, kernel_size, 2, 1)])
        if norm:
            self.layers.append(nn.InstanceNorm2d(op_sz))
        self.layers += [nn.LeakyReLU(0.2)]
        if dropout:
            self.layers += [nn.Dropout(dropout)]
    def forward(self, x):
        op = nn.Sequential(*(self.layers))(x)
        return op

In [ ]:
import torch.nn.utils.rnn as rnn

class SpeechEnhancementLSTM(nn.Module):
  #input size, hidden size, number of layers, output size, and a flag for using bidirectional LSTMs as its parameters
    def __init__(self, input_size, hidden_size, num_layers, output_size, bidirectional=True):
        super(SpeechEnhancementLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=bidirectional)
        self.fc = nn.Linear(hidden_size * 2 if bidirectional else hidden_size, output_size)

    def forward(self, x, lengths):
        x = rnn.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        x, _ = self.lstm(x)
        x, _ = rnn.pad_packed_sequence(x, batch_first=True)
        x = self.fc(x)
        return x

EHNET or CRN: Convoluation Encoder/Decoder with LSTM in the middle
Processing spectrogram inputs 

Paper: 
https://web.cse.ohio-state.edu/~wang.77/papers/Tan-Wang1.interspeech18.pdf

Code Reference:
https://github.com/haoxiangsnr/A-Convolutional-Recurrent-Neural-Network-for-Real-Time-Speech-Enhancement/blob/master/model/crn_in_paper_layer_norm.py

In [ ]:
import torch
import torch.nn as nn


class CausalConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=(3, 2),
            stride=(2, 1),
            padding=(0, 1)
        )
        self.norm = nn.BatchNorm2d(num_features=out_channels)
        self.activation = nn.ELU()

    def forward(self, x):
        """
        2D Causal convolution.
        Args:
            x: [B, C, F, T]
        Returns:
            [B, C, F, T]
        """
        x = self.conv(x)
        x = x[:, :, :, :-1]  # chomp size
        x = self.norm(x)
        x = self.activation(x)
        return x


class CausalTransConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, is_last=False, output_padding=(0, 0)):
        super().__init__()
        self.conv = nn.ConvTranspose2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=(3, 2),
            stride=(2, 1),
            output_padding=output_padding
        )
        self.norm = nn.BatchNorm2d(num_features=out_channels)
        if is_last:
            self.activation = nn.ReLU()
        else:
            self.activation = nn.ELU()

    def forward(self, x):
        """
        2D Causal convolution.
        Args:
            x: [B, C, F, T]
        Returns:
            [B, C, F, T]
        """
        x = self.conv(x)
        x = x[:, :, :, :-1]  # chomp size
        x = self.norm(x)
        x = self.activation(x)
        return x


class CRN(nn.Module):
    """
    Input: [batch size, channels=1, T, n_fft]
    Output: [batch size, T, n_fft]
    """

    def __init__(self):
        super(CRN, self).__init__()
        # Encoder
        self.conv_block_1 = CausalConvBlock(1, 16)
        self.conv_block_2 = CausalConvBlock(16, 32)
        self.conv_block_3 = CausalConvBlock(32, 64)
        self.conv_block_4 = CausalConvBlock(64, 128)
        self.conv_block_5 = CausalConvBlock(128, 256)

        # LSTM
        self.lstm_layer = nn.LSTM(input_size=1024, hidden_size=1024, num_layers=2, batch_first=True)

        self.tran_conv_block_1 = CausalTransConvBlock(256 + 256, 128)
        self.tran_conv_block_2 = CausalTransConvBlock(128 + 128, 64)
        self.tran_conv_block_3 = CausalTransConvBlock(64 + 64, 32)
        self.tran_conv_block_4 = CausalTransConvBlock(32 + 32, 16, output_padding=(1, 0))
        self.tran_conv_block_5 = CausalTransConvBlock(16 + 16, 1, is_last=True)

    def forward(self, x):
        self.lstm_layer.flatten_parameters()

        e_1 = self.conv_block_1(x)
        e_2 = self.conv_block_2(e_1)
        e_3 = self.conv_block_3(e_2)
        e_4 = self.conv_block_4(e_3)
        e_5 = self.conv_block_5(e_4)  # [2, 256, 4, 200]

        batch_size, n_channels, n_f_bins, n_frame_size = e_5.shape

        # [2, 256, 4, 200] = [2, 1024, 200] => [2, 200, 1024]
        lstm_in = e_5.reshape(batch_size, n_channels * n_f_bins, n_frame_size).permute(0, 2, 1)
        lstm_out, _ = self.lstm_layer(lstm_in)  # [2, 200, 1024]
        lstm_out = lstm_out.permute(0, 2, 1).reshape(batch_size, n_channels, n_f_bins, n_frame_size)  # [2, 256, 4, 200]

        d_1 = self.tran_conv_block_1(torch.cat((lstm_out, e_5), 1))
        d_2 = self.tran_conv_block_2(torch.cat((d_1, e_4), 1))
        d_3 = self.tran_conv_block_3(torch.cat((d_2, e_3), 1))
        d_4 = self.tran_conv_block_4(torch.cat((d_3, e_2), 1))
        d_5 = self.tran_conv_block_5(torch.cat((d_4, e_1), 1))

        return d_5

WaveU-Net for Speech Denoising

Code Reference:
https://github.com/haoxiangsnr/Wave-U-Net-for-Speech-Enhancement

Paper Reference:
https://arxiv.org/pdf/1806.03185.pdf


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class DownSamplingLayer(nn.Module):
    def __init__(self, channel_in, channel_out, dilation=1, kernel_size=15, stride=1, padding=7):
        super(DownSamplingLayer, self).__init__()
        self.main = nn.Sequential(
            nn.Conv1d(channel_in, channel_out, kernel_size=kernel_size,
                      stride=stride, padding=padding, dilation=dilation),
            nn.BatchNorm1d(channel_out),
            nn.LeakyReLU(negative_slope=0.1)
        )

    def forward(self, ipt):
        return self.main(ipt)

class UpSamplingLayer(nn.Module):
    def __init__(self, channel_in, channel_out, kernel_size=5, stride=1, padding=2):
        super(UpSamplingLayer, self).__init__()
        self.main = nn.Sequential(
            nn.Conv1d(channel_in, channel_out, kernel_size=kernel_size,
                      stride=stride, padding=padding),
            nn.BatchNorm1d(channel_out),
            nn.LeakyReLU(negative_slope=0.1, inplace=True),
        )

    def forward(self, ipt):
        return self.main(ipt)

class WaveUNet2(nn.Module):
    def __init__(self, n_layers=12, channels_interval=24):
        super(WaveUNet2, self).__init__()

        self.n_layers = n_layers
        self.channels_interval = channels_interval
        encoder_in_channels_list = [1] + [i * self.channels_interval for i in range(1, self.n_layers)]
        encoder_out_channels_list = [i * self.channels_interval for i in range(1, self.n_layers + 1)]

        #          1    => 2    => 3    => 4    => 5    => 6   => 7   => 8   => 9  => 10 => 11 =>12
        # 16384 => 8192 => 4096 => 2048 => 1024 => 512 => 256 => 128 => 64 => 32 => 16 =>  8 => 4
        self.encoder = nn.ModuleList()
        for i in range(self.n_layers):
            self.encoder.append(
                DownSamplingLayer(
                    channel_in=encoder_in_channels_list[i],
                    channel_out=encoder_out_channels_list[i]
                )
            )

        self.middle = nn.Sequential(
            nn.Conv1d(self.n_layers * self.channels_interval, self.n_layers * self.channels_interval, 15, stride=1,
                      padding=7),
            nn.BatchNorm1d(self.n_layers * self.channels_interval),
            nn.LeakyReLU(negative_slope=0.1, inplace=True)
        )

        decoder_in_channels_list = [(2 * i + 1) * self.channels_interval for i in range(1, self.n_layers)] + [
            2 * self.n_layers * self.channels_interval]
        decoder_in_channels_list = decoder_in_channels_list[::-1]
        decoder_out_channels_list = encoder_out_channels_list[::-1]
        self.decoder = nn.ModuleList()
        for i in range(self.n_layers):
            self.decoder.append(
                UpSamplingLayer(
                    channel_in=decoder_in_channels_list[i],
                    channel_out=decoder_out_channels_list[i]
                )
            )

        self.out = nn.Sequential(
            nn.Conv1d(1 + self.channels_interval, 1, kernel_size=1, stride=1),
            nn.Tanh()
        )

    def forward(self, input):
        tmp = []
        o = input

        # Up Sampling
        for i in range(self.n_layers):
            o = self.encoder[i](o)
            tmp.append(o)
            # [batch_size, T // 2, channels]
            o = o[:, :, ::2]

        o = self.middle(o)

        # Down Sampling
        for i in range(self.n_layers):
            # [batch_size, T * 2, channels]
            o = F.interpolate(o, scale_factor=2, mode="linear", align_corners=True)
            # Skip Connection
            o = torch.cat([o, tmp[self.n_layers - i - 1]], dim=1)
            o = self.decoder[i](o)

        o = torch.cat([o, input], dim=1)
        o = self.out(o)
        return o

Main block for training the model and testing.
Initializes all training and testing parameters. 
Performs training and testing in one big loop, testing each epoch to track any overfitting.

In [ ]:
# training (work in progress)
from torch.utils.data import DataLoader

def train_model( model, dataloader, criterion, optimizer, device ):
    model.train()

    running_loss = 0.0

    # Iterate over data.
    for clean_audio,noisy_audio,orig_len in dataloader:

        # send inputs to gpu
        clean_audio = clean_audio.to(device)
        noisy_audio = noisy_audio.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        
        with torch.enable_grad():
            # Send the noisey speech sample through network
            output = model(noisy_audio)

            # compute loss between network output and clean audio
            loss = criterion(output, clean_audio)

            # backward + optimize 
            loss.backward()
            optimizer.step()

        # statistics
        running_loss += loss.item()

    epoch_loss = running_loss / len(dataloader.dataset)

    return model, epoch_loss

def test_model( model, device, criterion, dataloader, inv_transform = None ):
    model.eval() 

    #cleaned_data = torch.empty(0)
    cleaned_data = []
    running_loss = 0

    for clean_audio, noisy_audio, orig_len in dataloader:
        #noisy_mag, noisy_phase, noisy_wav, norm_val = noisy_audio
        noisy_audio = noisy_audio.to(device)
       # clean_wav, clean_mag = clean_audio
        clean_audio = clean_audio.to(device)
        #norm_val = norm_val.to(device)
        
        with torch.no_grad():
            # forward
            enhanced_data = model(noisy_audio)

            loss = criterion(enhanced_data, clean_audio)

            #cleaned_data = torch.cat( (cleaned_data, enhanced_data.detach().cpu()), 0 )
            cleaned_data.append( enhanced_data.detach().cpu() )

            running_loss += loss.item()

    test_loss = running_loss / len( dataloader.dataset )

    return cleaned_data, test_loss


train_loader = DataLoader( train_set, batch_size=1)
test_loader = DataLoader( test_set, batch_size=1 )

model = CRN() # assign model you want to use

learning_rate = 0.01
num_epochs = 20

criterion = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(),learning_rate)

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")

model = model.to(device)

prev_test_loss = 10000.0
count = 0
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    model, epoch_loss = train_model( model, train_loader, criterion, optimizer, device )
    print('Train Epoch Loss = ', epoch_loss )

    cleaned_data, test_loss = \
        test_model( model, device, criterion, test_loader, inv_transform = None)
    print('Test Epoch Loss = ', test_loss )

    if prev_test_loss < test_loss:
        count += 1
    else:
        count = 0

    if count > 1:
        break;

    prev_test_loss = test_loss


In [ ]:
!pip install torchmetrics[audio]
!pip install pesq

Loop through testing samples and evaluate enhanced data against clean data

In [ ]:
from torchmetrics import SignalNoiseRatio
from pesq import pesq

SNR = SignalNoiseRatio(zero_mean=True)

snr_vals_noisy = torch.empty(len(test_set))
pesq_vals_noisy = torch.empty(len(test_set))
snr_vals = torch.empty(len(test_set))
pesq_vals = torch.empty(len(test_set))
cleaned_waveforms = []
#inv_transform = IDWT( name = 'db8', level = 5)
inv_transform = ta.transforms.InverseSpectrogram( 
    n_fft = 321,
    normalized = False)

complex_spec = ta.transforms.Spectrogram(
    n_fft=321,
    power=None,
    normalized = False ) # return complex spectrum

test_complex = VoiceBankDemand( clean_dir = test_clean_path,
                             noisy_dir = test_noisy_path,
                             list_dir = list_dir_test,
                             data = "test",
                             len_samples = None,
                             downsample = target_fs,
                             transform = complex_spec )

test_wavs = VoiceBankDemand( clean_dir = test_clean_path,
                             noisy_dir = test_noisy_path,
                             list_dir = list_dir_test,
                             data = "test",
                             len_samples = None,
                             downsample = target_fs,
                             transform = None )

for i in range( len(test_wavs) ):
    clean_wav,noisy_wav,orig_len = test_wavs[i]
    enhanced_audio = cleaned_data[i]

    enhanced_audio = enhanced_audio.detach().cpu()

    if inv_transform:
        _,noisy_complex, orig_len = test_complex[i]
        phase = torch.angle(noisy_complex)
        complex_audio = torch.polar(enhanced_audio.squeeze(), phase)
        enhanced_audio = inv_transform( complex_audio, clean_wav.shape[1] )

    if enhanced_audio.shape[1] > orig_len:
        enhanced_audio = enhanced_audio[:,:orig_len]
        noisy_wav = noisy_wav[:,:orig_len]
        clean_wav = clean_wav[:,:orig_len]

    cleaned_waveforms.append( enhanced_audio )

    snr_vals[i] = SNR(enhanced_audio, clean_wav)
    pesq_vals[i] = pesq( target_fs, enhanced_audio.squeeze().numpy(), clean_wav.squeeze().numpy(), "nb")

    snr_vals_noisy[i] = SNR(noisy_wav, clean_wav)
    pesq_vals_noisy[i] = pesq( target_fs, noisy_wav.squeeze().numpy(), clean_wav.squeeze().numpy(), "nb")



Compute average and best SNR and PESQ improvements

In [ ]:
SNR_imp = snr_vals - snr_vals_noisy
PESQ_imp = pesq_vals - pesq_vals_noisy
print("Average PESQ Enhanced", pesq_vals.sum() / len(pesq_vals) )
print("Average PESQ Noisy", pesq_vals_noisy.sum() / len(pesq_vals_noisy) )

avg_SNR_imp = SNR_imp.sum() / len(SNR_imp)
avg_PESQ_imp = PESQ_imp.sum() / len(PESQ_imp)

peak_SNR_imp = SNR_imp.max()
peak_snr_ind = SNR_imp.argmax()
peak_PESQ_imp = PESQ_imp.max()
peak_PESQ_ind = PESQ_imp.argmax()


print("Average SNR Improvement = ", avg_SNR_imp )
print("Average PESQ Improvement = ", avg_PESQ_imp )

print("Best SNR Improvement = ", peak_SNR_imp )
print("Best PESQ Improvment = ", peak_PESQ_imp )


# use if using magnitude spectrograms
#plot_spectrogram(cleaned_data[0].squeeze())
#plot_spectrogram(test_set[0][0][1].squeeze())

play_audio(test_wavs[800][1],target_fs) # play noisy test sample
play_audio(cleaned_waveforms[800],target_fs)


In [ ]:
print( snr_vals_noisy.argmin() )
print(pesq_vals_noisy[588] )